In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [2]:
df = pd.read_csv ('data_Ithaca.csv')
data = df.values
# 13th column for ground truth load, 29th column for disaggregated load
names = df.columns[[13,10,23,24,25,26,6,27,0,2,22]]
print(names)
data = data[:,[13,10,23,24,25,26,6,27,0,2,22]]
mean_c, std_c, min_c, max_c = np.mean(data[:,0]), np.std(data[:,0]), np.min(data[:,0]), np.max(data[:,0])
print('mean',mean_c, 'std',std_c, 'max',max_c, 'min',min_c)
# normalize entire dataset to [0,1]
data[:,0] = (data[:,0]-min_c)/(max_c-min_c)
data = data.astype('float32')
for i in [1,2,3,4,5,6,7]:
    min_temp, max_temp = np.min(data[:,i]), np.max(data[:,i]) 
    data[:,i] = (data[:,i]-min_temp)/(max_temp-min_temp)
print(data.shape)

Index(['consumption', 'Temperature', 'Dew Point', 'Wind Speed.1',
       'Precipitable Water', 'Relative humidity', 'zenith', 'Pressure',
       'weekday', 'timeslot of day', 'Cloud Type'],
      dtype='object')
mean 10.385668138586956 std 4.49667223574142 max 37.726 min 3.298
(17664, 11)


In [3]:
# employ sliding window to stack data correspondingly
def Create_dataset(dataset):
    data_X, data_Y, data_Z, data_W = [], [], [], []
    dt = dataset.tolist()
    for i in range(len(dataset)-820): 
        indice = list(range(i,i+96))+list(range(i+96*5,i+96*7))
        tempx = np.array([dt[index] for index in indice])
        tempy = dataset[i+724:i+820,:]
        tempw = dataset[i:i+96,:]
        temp1 = tempx[:,0].reshape(-1,1)
        temp2 = tempy[:,1:8].reshape(-1,1)
        temp3 = np.concatenate((temp1,temp2)).tolist()
        temp4 = tempy[:,0].tolist()
        temp5 = tempy[:,8:].reshape(-1,1)
        temp6 = tempw[:,0].reshape(-1,1)
       
        data_W.append(temp6) # load traces from the day one week ago
        data_X.append(temp3) # load traces from the past two days and numerical weather forecast
        data_Y.append(temp4) # targeted load traces (13 hours ahead ~ 37 hours ahead)
        data_Z.append(temp5) # categorical features
    
    return np.array(data_X), np.array(data_Y), np.array(data_Z), np.array(data_W)

In [4]:
# encode categorical input features
one_hot = lambda label,num_classes: F.one_hot(label.long(), num_classes=num_classes).type(torch.float32)
cycl_ = lambda x,num_classes : torch.tensor((np.sin(x / num_classes * 2 * np.pi),np.cos(x / num_classes * 2 * np.pi))).type(torch.float32)

def Calender(data):
    calender = []
    for i in range(data.shape[0]):
        ty = data[i]
        temp = torch.zeros(96,19)
        for j in range(96):
            W = one_hot(ty[3*j]-1,7)                   # weekday
            H = torch.unsqueeze(cycl_(ty[3*j+1],24),0) # timeslot of the day
            C = one_hot(ty[3*j+2],10)                  # cloud type
            temp[j,:] = torch.cat((W,H,C),1)
        temp = temp.reshape(1,-1)  
        calender.append(temp.tolist())
    return calender

In [5]:
# evenly divide the whole dataset into five folds for cross validation
test_indice1 = list(range(0,672))+list(range(3360,3360+672))+list(range(3360*2,3360*2+672))+list(range(3360*3,3360*3+672))+list(range(3360*4,3360*4+672))
test_indice2 = list(range(672,672*2))+list(range(3360+672,3360+672*2))+list(range(3360*2+672,3360*2+672*2))+list(range(3360*3+672,3360*3+672*2))+list(range(3360*4+672,3360*4+672*2))
test_indice3 = list(range(672*2,672*3))+list(range(3360+672*2,3360+672*3))+list(range(3360*2+672*2,3360*2+672*3))+list(range(3360*3+672*2,3360*3+672*3))+list(range(3360*4+672*2,3360*4+672*3))
test_indice4 = list(range(672*3,672*4))+list(range(3360+672*3,3360+672*4))+list(range(3360*2+672*3,3360*2+672*4))+list(range(3360*3+672*3,3360*3+672*4))+list(range(3360*4+672*3,3360*4+672*4))
test_indice5 = list(range(672*4,672*5))+list(range(3360+672*4,3360+672*5))+list(range(3360*2+672*4,3360*2+672*5))+list(range(3360*3+672*4,3360*3+672*5))+list(range(3360*4+672*4,3360*4+672*5))
train_indice = list(range(0,16844))

In [6]:
# generate input features data_X and output labels data_Y
data_X, data_Y, data_Z, data_W = Create_dataset(data)
data_Y = np.expand_dims(data_Y,2)
data_Z = np.array(Calender(torch.tensor(data_Z)))
data_Z = np.transpose(data_Z,(0,2,1))
data_X = np.concatenate((data_W,data_X,data_Z),1)

In [7]:
# generate training data and testing data respectively for each one of the five folds
train_X, test_X = [data_X[index] for index in train_indice if index not in test_indice1], [data_X[index] for index in test_indice1]
train_Y, test_Y = [data_Y[index] for index in train_indice if index not in test_indice1], [data_Y[index] for index in test_indice1]
train_X, train_Y = np.array(train_X), np.array(train_Y)
test_X, test_Y = np.array(test_X), np.array(test_Y)

In [8]:
class Train(Dataset):
    def __init__(self, data):
        self.skip, self.data, self.weather, self.calender, self.label = data[:,:96,:].float(), data[:,96:384,:].float(), data[:,384:1056,:].float(), data[:,1056:2880,:].float(), data[:,-96:,:].float()

    def __getitem__(self, index):
        return self.skip[index], self.data[index], self.weather[index], self.calender[index], self.label[index]

    def __len__(self):
        return len(self.data)

In [9]:
# load training data into DataLoader
train_loader = DataLoader(Train(torch.cat((torch.tensor(train_X),torch.tensor(train_Y)),1)), batch_size=500, shuffle=True)

In [10]:
# FCNN model
class ANN(nn.Module):
    
    def __init__(self):
        super(ANN, self).__init__()
        
        self.ann = nn.Sequential(
            nn.Linear(288,576),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.LayerNorm(576),
        )
        self.categorical = nn.Sequential(
            nn.Linear(26,1),
        )
        self.skip = nn.Sequential(
            nn.Linear(96,576),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.LayerNorm(576),
        )
        self.out = nn.Sequential(
            nn.Linear(576,96),
            nn.Dropout(p=0.1),
            nn.LayerNorm(96),
        )
        self.relu = nn.ReLU()

    def forward(self, k, x, y, w):
        
        batch = x.shape[0]
        tk = self.skip(k)
        tx = self.ann(x)
        tx = tx+tk
        ty = torch.reshape(y,(batch,96,-1))
        tw = torch.reshape(w,(batch,96,-1))
        twy = torch.cat((tw,ty),2)
        twy = self.categorical(twy)
        twy  = torch.transpose(twy,1,2)
        tx = self.out(tx)
        out = tx+twy
        # use relu at final step to generate positive output
        out = self.relu(out)  
        return out

In [30]:
# LSTM model
class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=1,   
            hidden_size=20,
            num_layers=2, 
            batch_first=True,
            dropout=0.1,
        )
        self.categorical = nn.Sequential(
            nn.Linear(26,1),
        )
        self.out = nn.Sequential(
            nn.Linear(20,1),
        )
        self.state = None
        self.relu = nn.ReLU()

    def forward(self, k, x, y, w):
        
        batch = x.shape[0]
        tx, self.state = self.lstm(x, state)
        ty = torch.reshape(y,(batch,96,-1))
        tw = torch.reshape(w,(batch,96,-1))
        twy = torch.cat((tw,ty),2)
        temp = self.categorical(twy)
        tx = self.out(tx[:,-96:,:])
        out = tx+temp
        # use relu at final step to generate positive output 
        out = self.relu(out)
        return out

In [11]:
# hyper-parameters for training process
LR = 0.01
EPOCH = 500
Loss = []
best_loss = 100
state = None
cal_loss = nn.MSELoss()

In [12]:
# training FCNN model
model = ANN()
optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
for i in range(EPOCH):
    for j, entry in enumerate(train_loader):
        tk, tx, ty, tw, tz = entry
        tx = torch.transpose(tx,1,2)
        ty = torch.transpose(ty,1,2)
        tw = torch.transpose(tw,1,2)
        tk = torch.transpose(tk,1,2)
        tz = torch.transpose(tz,1,2)
        final_out = model(tk,tx,ty,tw)
        loss = cal_loss(final_out, tz)
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step()
        Loss.append(loss.detach().numpy())
    print('epoch{}'.format(i+1), loss.detach().numpy())
    if loss.detach().numpy() < best_loss:
        best_loss = loss.detach().numpy()
        torch.save(model, 'ithaca_fcnn_load'.format(loss.detach().numpy()))
        print('new fcnn saved at epoch {} with loss {}'.format(i+1, best_loss))

epoch1 0.055029094
new fcnn saved at epoch 1 with loss 0.055029094219207764
epoch2 0.055427447
epoch3 0.05064194
new fcnn saved at epoch 3 with loss 0.05064193904399872
epoch4 0.047128253
new fcnn saved at epoch 4 with loss 0.04712825268507004
epoch5 0.04629001
new fcnn saved at epoch 5 with loss 0.04629001021385193
epoch6 0.042683546
new fcnn saved at epoch 6 with loss 0.042683545500040054
epoch7 0.04152984
new fcnn saved at epoch 7 with loss 0.04152984172105789
epoch8 0.03831702
new fcnn saved at epoch 8 with loss 0.03831702098250389
epoch9 0.037422366
new fcnn saved at epoch 9 with loss 0.03742236644029617
epoch10 0.03453983
new fcnn saved at epoch 10 with loss 0.034539829939603806
epoch11 0.032338835
new fcnn saved at epoch 11 with loss 0.032338835299015045
epoch12 0.032800484
epoch13 0.030929783
new fcnn saved at epoch 13 with loss 0.03092978335916996
epoch14 0.031030685
epoch15 0.029298244
new fcnn saved at epoch 15 with loss 0.029298244044184685
epoch16 0.028805098
new fcnn save

KeyboardInterrupt: ignored

In [ ]:
# training LSTM model
model = LSTM()
optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
for i in range(EPOCH):
    for j, entry in enumerate(train_loader):
        tk, tx, ty, tw, tz = entry
        if state is not None:
            state = state.detach()
        final_out = model(tx,ty,tw)
        loss = cal_loss(final_out, tz)
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step()
        Loss.append(loss.detach().numpy())
    print('epoch{}'.format(i+1), loss.detach().numpy())
    if loss.detach().numpy() < best_loss:
        best_loss = loss.detach().numpy()
        torch.save(model, 'ithaca_lstm_load'.format(loss.detach().numpy()))
        print('new lstm saved at epoch {} with loss {}'.format(i+1, best_loss))

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(Loss,'b')
plt.title('Training Loss for Load',fontsize=15)
plt.savefig('training_loss.png')

In [47]:
# generate prediction for testing set with trained fcnn model
model = torch.load('ithaca_fcnn_load')
model.eval()
test_res = test_X[:,:96,:]
test_res = torch.tensor(test_res, dtype=torch.float32)
test_res = torch.transpose(test_res,1,2)
test_load = test_X[:,96:384,:]
test_load = torch.tensor(test_load, dtype=torch.float32)
test_load = torch.transpose(test_load,1,2)
test_weather = test_X[:,384:1344,:]
test_weather = torch.tensor(test_weather, dtype=torch.float32)
test_weather = torch.transpose(test_weather,1,2)
test_date = test_X[:,1344:,:]
test_date = torch.tensor(test_date, dtype=torch.float32)
prediction = model(test_res,test_load,test_weather,test_date)
torch.save(prediction, 'prediction_load_fcnn.pt')

In [48]:
# generate prediction for testing set with trained lstm model
model = torch.load('ithaca_lstm_load')
model.eval()
state = None
test_res = test_X[:,:96,:]
test_res = torch.tensor(test_res, dtype=torch.float32)
test_load = test_X[:,96:384,:]
test_load = torch.tensor(test_load, dtype=torch.float32)
test_weather = test_X[:,384:1344,:]
test_weather = torch.tensor(test_weather, dtype=torch.float32)
test_date = test_X[:,1344:,:]
test_date = torch.tensor(test_date, dtype=torch.float32)
prediction = model(test_res, test_load, test_weather, test_date)
torch.save(prediction, 'prediction_load_lstm.pt')